# Using Optimization Techniques to Generate Winning FanDuel Lineups
___
Until I build one from scratch I'm going to be testing out `pydfs-lineup-optimizer` to generate optimal lineups with projections that I already have. More information about `pydfs-lineup-optimizer` can be found [here](https://github.com/DimaKudosh/pydfs-lineup-optimizer/tree/master/pydfs_lineup_optimizer/sites/fanduel).

**Here is one quick example of generating lineups based on average points per game.**

In [27]:
import pandas as pd
from pydfs_lineup_optimizer import get_optimizer, Site, Sport, CSVLineupExporter

In [28]:
optimizer = get_optimizer(Site.FANDUEL, Sport.FOOTBALL)

optimizer.load_players_from_csv('player-lists/Week-13/FanDuel-NFL-2018-12-02-30514-players-list.csv')
for lineup in optimizer.optimize(n=5, max_exposure=0.3):
    print(lineup.players)
    print(lineup.fantasy_points_projection)
    print(lineup.salary_costs)

[Ryan Fitzpatrick QB (TB), Todd Gurley II RB (LAR), James White RB (NE), Tyreek Hill WR (KC), John Brown WR (BAL), Leonte Carroo WR (MIA), Travis Kelce TE (KC), Minnesota Vikings D (MIN), Saquon Barkley RB (NYG)]
149.521
60000.0
[Ryan Fitzpatrick QB (TB), Todd Gurley II RB (LAR), T.J. Yeldon RB (JAC), Tyreek Hill WR (KC), John Brown WR (BAL), Leonte Carroo WR (MIA), Travis Kelce TE (KC), Chicago Bears D (CHI), Saquon Barkley RB (NYG)]
149.185
59500.0
[Patrick Mahomes QB (KC), T.J. Yeldon RB (JAC), Christian McCaffrey RB (CAR), Davante Adams WR (GB), Sterling Shepard WR (NYG), Emmanuel Sanders WR (DEN), Nick O'Leary TE (MIA), Chicago Bears D (CHI), James White RB (NE)]
138.664
59800.0
[Patrick Mahomes QB (KC), Kareem Hunt RB (KC), Joe Mixon RB (CIN), Jarvis Landry WR (CLE), Sterling Shepard WR (NYG), Emmanuel Sanders WR (DEN), Nick O'Leary TE (MIA), Minnesota Vikings D (MIN), Christian McCaffrey RB (CAR)]
133.31
60000.0
[Jared Goff QB (LAR), Kareem Hunt RB (KC), Mike Davis RB (SEA), Dav

In [29]:
problematic_names_dict = {'Melvin Gordon':'Melvin Gordon III', 'Odell Beckham':'Odell Beckham Jr.',\
                          'Will Fuller':'Will Fuller V', 'Marvin Jones':'Marvin Jones Jr.',\
                          'Willie Snead':'Willie Snead IV', 'Ted Ginn':'Ted Ginn Jr.', \
                          'Allen Robinson':'Allen Robinson II', 'Patrick Mahomes II':'Patrick Mahomes'}

## Now instead of just using the average points per game, we'll bring in our other projections that *should* be more accurate.

___
#### 1. Ballers

In [30]:
current_week = 13

qbs = pd.read_csv('projections/Week-{}/DFS - Fanduel QB.csv'.format(current_week))
rbs = pd.read_csv('projections/Week-{}/DFS - Fanduel RB.csv'.format(current_week))
wrs = pd.read_csv('projections/Week-{}/DFS - Fanduel WR.csv'.format(current_week))
tes = pd.read_csv('projections/Week-{}/DFS - Fanduel TE.csv'.format(current_week))
dst = pd.read_csv('projections/Week-{}/DFS - Fanduel DST.csv'.format(current_week))
full_team_names = list(dst.sort_values('Player')['Player'])

In [31]:
footballers = pd.concat([qbs,rbs,wrs,tes,dst])
footballers.rename(columns={"Player": "Nickname"}, inplace=True)
footballers.Nickname.replace(problematic_names_dict, inplace=True)

___
#### 2. Sportsline

In [32]:
sportsline = pd.read_csv('projections/Week-{}/projections.csv'.format(current_week))
sportsline.rename(columns={"PLAYER": "Nickname"}, inplace=True)
sportsline.Nickname.replace(problematic_names_dict, inplace=True)
abbv_team_names = list(sportsline.loc[sportsline.POS == 'DEF', :].sort_values('Nickname')['Nickname'])
defenses_dict = dict(zip(abbv_team_names, full_team_names))
sportsline['Nickname'].replace(defenses_dict, inplace = True)
sportsline.head(2)

,Nickname,POS,TEAM,OPP,FP,fd exp,dk exp,PPR,CONS%,FREB%,PASSYD,RUSHYD,RECYD
0,Patrick Mahomes,QB,KC,@OAK,35.4,24.6,28.4,35.4,60%,50%,379.0,18.0,NaN
1,Cam Newton,QB,CAR,@TB,28.1,22.5,24.7,28.1,49%,53%,300.0,42.0,NaN


___
#### 3. FantasyPros

In [33]:
fp_qb = pd.read_csv('projections/Week-{}/FantasyPros_Fantasy_Football_Projections_QB.csv'.format(current_week))
fp_rb = pd.read_csv('projections/Week-{}/FantasyPros_Fantasy_Football_Projections_RB.csv'.format(current_week))
fp_wr = pd.read_csv('projections/Week-{}/FantasyPros_Fantasy_Football_Projections_WR.csv'.format(current_week))
fp_te = pd.read_csv('projections/Week-{}/FantasyPros_Fantasy_Football_Projections_TE.csv'.format(current_week))
fp_dst = pd.read_csv('projections/Week-{}/FantasyPros_Fantasy_Football_Projections_DST.csv'.format(current_week))

In [34]:
for df in [fp_qb, fp_rb, fp_wr, fp_te, fp_dst]:
    df.drop([col for col in df.columns if col not in ['Player', 'FPTS']], axis=1, inplace=True)

In [35]:
fantasy_pros = pd.concat([fp_qb, fp_rb, fp_wr, fp_te, fp_dst])
fantasy_pros.rename(columns={"Player": "Nickname"}, inplace=True)
fantasy_pros.Nickname.replace(problematic_names_dict, inplace=True)

___
#### Averaging the projections:

In [36]:
total_projections = pd.merge(footballers, sportsline, on="Nickname")
total_projections = pd.merge(total_projections, fantasy_pros, on="Nickname")
total_projections['Salary'] = total_projections.Salary.apply(lambda x: int(x.replace(',', '')))
total_projections['Pts'] = (total_projections.Pts + total_projections.FP + total_projections.FPTS) / 3
total_projections['Pts Per 1k'] = total_projections['Pts'] / (total_projections.Salary / 1000)
total_projections.drop(labels=list(total_projections.columns[5:]), axis=1, inplace=True)
total_projections.head(3)

,Nickname,Opp,Pts,Salary,Pts Per 1k
0,Cam Newton,at TB,26.066667,8700,2.996169
1,Jameis Winston,vs. CAR,21.133333,7500,2.817778
2,Jared Goff,at DET,25.133333,8400,2.992063


In [37]:
total_projections.loc[total_projections.Nickname == 'Matt Breida', :]

,Nickname,Opp,Pts,Salary,Pts Per 1k
41,Matt Breida,at SEA,12.766667,6400,1.994792


___
#### Bringing in the player list from FanDuel:

In [38]:
# Make sure you're selecting the correct file:
player_list=pd.read_csv('player-lists/Week-13/FanDuel-NFL-2018-12-02-30514-players-list.csv')
player_list.head(3)

,Id,Position,First Name,Nickname,Last Name,FPPG,Played,Salary,Game,Team,Opponent,Injury Indicator,Injury Details,Tier,Unnamed: 14,Unnamed: 15
0,30514-30447,RB,Todd,Todd Gurley II,Gurley II,25.081818,11,9800,LAR@DET,LAR,DET,NaN,NaN,NaN,NaN,NaN
1,30514-57439,QB,Patrick,Patrick Mahomes,Mahomes,28.156364,11,9500,KC@OAK,KC,OAK,NaN,NaN,NaN,NaN,NaN
2,30514-45229,RB,Kareem,Kareem Hunt,Hunt,19.745454,11,8900,KC@OAK,KC,OAK,NaN,NaN,NaN,NaN,NaN


In [39]:
player_list.loc[player_list.Nickname == 'Matt Breida', :]

,Id,Position,First Name,Nickname,Last Name,FPPG,Played,Salary,Game,Team,Opponent,Injury Indicator,Injury Details,Tier,Unnamed: 14,Unnamed: 15
63,30514-53303,RB,Matt,Matt Breida,Breida,11.509091,11,6400,SF@SEA,SF,SEA,Q,Ankle,NaN,NaN,NaN


> Merging those 2 and making the swap:

In [40]:
master_projections = pd.merge(left=player_list, right=total_projections, how='inner', on='Nickname', suffixes=['_FD', '_tp'])
master_projections['FPPG'] = master_projections['Pts Per 1k'] # Experiment with Pts OR Pts Per 1k
master_projections.drop(['Opp', 'Pts', 'Salary_tp', 'Pts Per 1k'],axis=1, inplace=True)
master_projections.rename(columns={'Salary_FD':'Salary'}, inplace=True)
master_projections.to_csv('projections/Week-{}/master_FD_projections.csv'.format(current_week))
master_projections.head(3)

,Id,Position,First Name,Nickname,Last Name,FPPG,Played,Salary,Game,Team,Opponent,Injury Indicator,Injury Details,Tier,Unnamed: 14,Unnamed: 15
0,30514-57439,QB,Patrick,Patrick Mahomes,Mahomes,3.115789,11,9500,KC@OAK,KC,OAK,NaN,NaN,NaN,NaN,NaN
1,30514-16606,WR,DeAndre,DeAndre Hopkins,Hopkins,2.204545,11,8800,CLE@HOU,HOU,CLE,Q,Foot,NaN,NaN,NaN
2,30514-55050,RB,Christian,Christian McCaffrey,McCaffrey,2.549242,11,8800,CAR@TB,CAR,TB,NaN,NaN,NaN,NaN,NaN


In [41]:
master_projections.loc[master_projections.Position == 'D', :].sort_values('FPPG', ascending= False)[:5]

,Id,Position,First Name,Nickname,Last Name,FPPG,Played,Salary,Game,Team,Opponent,Injury Indicator,Injury Details,Tier,Unnamed: 14,Unnamed: 15
200,30514-12538,D,Los Angeles,Los Angeles Rams,Rams,3.032520,11,4100,LAR@DET,LAR,DET,NaN,NaN,NaN,NaN,NaN
139,30514-12527,D,Chicago,Chicago Bears,Bears,2.881944,11,4800,CHI@NYG,CHI,NYG,NaN,NaN,NaN,NaN,NaN
159,30514-12550,D,Seattle,Seattle Seahawks,Seahawks,2.782609,11,4600,SF@SEA,SEA,SF,NaN,NaN,NaN,NaN,NaN
218,30514-12537,D,Oakland,Oakland Raiders,Raiders,2.650000,11,2000,KC@OAK,OAK,KC,NaN,NaN,NaN,NaN,NaN
216,30514-12532,D,Detroit,Detroit Lions,Lions,2.388889,11,3000,LAR@DET,DET,LAR,NaN,NaN,NaN,NaN,NaN


In [42]:
master_projections.loc[(master_projections.Nickname == 'Adam Thielen'), :]

,Id,Position,First Name,Nickname,Last Name,FPPG,Played,Salary,Game,Team,Opponent,Injury Indicator,Injury Details,Tier,Unnamed: 14,Unnamed: 15
13,30514-39716,WR,Adam,Adam Thielen,Thielen,2.053498,11,8100,MIN@NE,MIN,NE,Q,Calf,NaN,NaN,NaN


___
# Alright! Now let's just use our optimizer tools from above to generate some better lineups

In [43]:
out_players = list(master_projections.loc[(master_projections['Injury Indicator'] == 'IR') | (master_projections['Injury Indicator'] == 'O'), :]['Nickname'].values)
low_scorers = list(master_projections.loc[master_projections['FPPG'] < 0.80, :]['Nickname'].values)

all_removals = out_players + low_scorers
all_removals = list(set(all_removals))

In [46]:
optimizer = get_optimizer(Site.FANDUEL, Sport.FOOTBALL)

optimizer.load_players_from_csv('projections/Week-{}/master_FD_projections.csv'.format(current_week))

# Removing certain players
for each_player in all_removals:
    optimizer.remove_player(each_player)

# Setting positional constraints
# optimizer.set_players_with_same_position({'TE': 0})
# No stacking on this one

# Making more randomness
optimizer.set_deviation(0.1, .24)

# Setting a minimum salary cap
optimizer.set_min_salary_cap(58000)

# Generating 20 lineups
for lineup in optimizer.optimize(n=7, max_exposure=.35):
    print(lineup, '\n')

 1. QB    Russell Wilson                QB    SEA            2.896   7700.0$   
 2. RB    Spencer Ware                  RB    KC             2.904   5200.0$   
 3. RB    Christian McCaffrey           RB    CAR            2.549   8800.0$   
 4. WR    Brandin Cooks                 WR    LAR            2.06    7800.0$   
 5. WR    Tyreek Hill                   WR    KC             2.472   8400.0$   
 6. WR    Chris Godwin                  WR    TB             1.833   5400.0$   
 7. TE    Cameron Brate                 TE    TB             1.932   4900.0$   
 8. D     Los Angeles Rams              D     LAR            3.033   4100.0$   
 9. FLEX  Aaron Jones                   RB    GB             2.553   7600.0$   

Fantasy Points 22.232
Salary 59900.0 

 1. QB    Russell Wilson                QB    SEA            2.896   7700.0$   
 2. RB    Spencer Ware                  RB    KC             2.904   5200.0$   
 3. RB    Christian McCaffrey           RB    CAR            2.549   8800.0$   


In [61]:
exporter = CSVLineupExporter(optimizer.optimize(n=20, max_exposure=.25))
exporter.export('submissions/Week-{}/fanduel-lineups-nostacks.csv'.format(current_week))

In [102]:
optimizer = get_optimizer(Site.FANDUEL, Sport.FOOTBALL)

optimizer.load_players_from_csv('projections/Week-{}/master_FD_projections.csv'.format(current_week))

# Removing certain players
for each_player in all_removals:
    optimizer.remove_player(each_player)

# Setting positional constraints
optimizer.set_players_with_same_position({'TE': 0})
optimizer.set_positions_for_same_team(['QB', 'WR'])

# Making more randomness
optimizer.set_deviation(0.06, .24)

# Setting a minimum salary cap
optimizer.set_min_salary_cap(58000)

# Capping the number of repeating players
optimizer.set_max_repeating_players(5)

# Generating 20 lineups
for lineup in optimizer.optimize(n=30, max_exposure=.35):
    print(lineup, '\n')

 1. QB    Jared Goff                    QB    LAR            3.056   8400.0$   
 2. RB    Carlos Hyde                   RB    JAC            1.893   5000.0$   
 3. RB    Aaron Jones                   RB    GB             2.43    7600.0$   
 4. WR    Robert Woods                  WR    LAR            2.091   7300.0$   
 5. WR    Tyreek Hill                   WR    KC             2.448   8400.0$   
 6. WR    Jarvis Landry                 WR    CLE            1.806   5500.0$   
 7. TE    Cameron Brate                 TE    TB             1.932   4900.0$   
 8. D     Los Angeles Rams              D     LAR            2.935   4100.0$   
 9. FLEX  Christian McCaffrey           RB    CAR            2.489   8800.0$   

Fantasy Points 21.08
Salary 60000.0 

 1. QB    Jared Goff                    QB    LAR            3.056   8400.0$   
 2. RB    Aaron Jones                   RB    GB             2.43    7600.0$   
 3. RB    Phillip Lindsay               RB    DEN            2.162   7000.0$   
 

KeyboardInterrupt: 

In [63]:
exporter = CSVLineupExporter(optimizer.optimize(n=20, max_exposure=.25))
exporter.export('submissions/Week-{}/fanduel-lineups-qbwr.csv'.format(current_week))

In [64]:
# Doing the second half with a RB + D stack

optimizer = get_optimizer(Site.FANDUEL, Sport.FOOTBALL)

optimizer.load_players_from_csv('projections/Week-{}/master_FD_projections.csv'.format(current_week))

# Removing certain players
for each_player in all_removals:
    optimizer.remove_player(each_player)

# Setting positional constraints
optimizer.set_players_with_same_position({'TE': 0})
optimizer.set_positions_for_same_team(['RB', 'D'])

    
# Making more randomness
optimizer.set_deviation(0.06, 0.24)

# Setting a minimum salary cap
optimizer.set_min_salary_cap(58000)

# Generating 20 lineups
for lineup in optimizer.optimize(n=20, max_exposure=.25):
    print(lineup, '\n')

 1. QB    Andrew Luck                   QB    IND            2.552   8400.0$   
 2. RB    James Conner                  RB    PIT            2.008   7900.0$   
 3. RB    Matt Breida                   RB    SF             1.948   6400.0$   
 4. WR    Mike Evans                    WR    TB             1.696   7900.0$   
 5. WR    Antonio Brown                 WR    PIT            1.855   8500.0$   
 6. WR    Jarvis Landry                 WR    CLE            1.683   6100.0$   
 7. TE    Cameron Brate                 TE    TB             1.765   4400.0$   
 8. D     Baltimore Ravens              D     BAL            2.707   5000.0$   
 9. FLEX  Gus Edwards                   RB    BAL            1.723   5300.0$   

Fantasy Points 17.937
Salary 59900.0 

 1. QB    Andrew Luck                   QB    IND            2.552   8400.0$   
 2. RB    James Conner                  RB    PIT            2.008   7900.0$   
 3. RB    Matt Breida                   RB    SF             1.948   6400.0$   


 1. QB    Lamar Jackson                 QB    BAL            2.311   7400.0$   
 2. RB    Saquon Barkley                RB    NYG            1.941   9000.0$   
 3. RB    Marlon Mack                   RB    IND            1.805   7000.0$   
 4. WR    Quincy Enunwa                 WR    NYJ            1.44    5000.0$   
 5. WR    Keenan Allen                  WR    LAC            1.671   7700.0$   
 6. WR    Kenny Stills                  WR    MIA            1.447   5300.0$   
 7. TE    Eric Ebron                    TE    IND            1.406   5500.0$   
 8. D     Jacksonville Jaguars          D     JAC            2.569   4800.0$   
 9. FLEX  Leonard Fournette             RB    JAC            1.568   8100.0$   

Fantasy Points 16.157
Salary 59800.0 

 1. QB    Ryan Tannehill                QB    MIA            2.197   6600.0$   
 2. RB    Saquon Barkley                RB    NYG            1.941   9000.0$   
 3. RB    Christian McCaffrey           RB    CAR            1.905   8400.0$   


In [65]:
exporter = CSVLineupExporter(optimizer.optimize(n=20, max_exposure=.25))
exporter.export('submissions/Week-{}/fanduel-lineups-rbdst.csv'.format(current_week))

In [92]:
# QB + TE stack

optimizer = get_optimizer(Site.FANDUEL, Sport.FOOTBALL)

optimizer.load_players_from_csv('projections/Week-{}/master_FD_projections.csv'.format(current_week))

# Removing certain players
for each_player in all_removals:
    optimizer.remove_player(each_player)

# Setting positional constraints
optimizer.set_players_with_same_position({'TE': 0})
optimizer.set_positions_for_same_team(['QB', 'TE'])

# Making more randomness
optimizer.set_deviation(0.06, 0.24)

# Setting a minimum salary cap
# optimizer.set_min_salary_cap(57000)

# Generating 20 lineups
for lineup in optimizer.optimize(n=5, max_exposure=.35):
    print(lineup, '\n')

 1. QB    Jameis Winston                QB    TB             2.436   7500.0$   
 2. RB    James Conner                  RB    PIT            2.008   7900.0$   
 3. RB    Joe Mixon                     RB    CIN            1.84    7300.0$   
 4. WR    Antonio Brown                 WR    PIT            1.855   8500.0$   
 5. WR    Tyler Boyd                    WR    CIN            1.593   6800.0$   
 6. WR    Jarvis Landry                 WR    CLE            1.683   6100.0$   
 7. TE    Cameron Brate                 TE    TB             1.765   4400.0$   
 8. D     Baltimore Ravens              D     BAL            2.707   5000.0$   
 9. FLEX  Matt Breida                   RB    SF             1.948   6400.0$   

Fantasy Points 17.835
Salary 59900.0 

 1. QB    Jameis Winston                QB    TB             2.436   7500.0$   
 2. RB    James Conner                  RB    PIT            2.008   7900.0$   
 3. RB    Matt Breida                   RB    SF             1.948   6400.0$   


In [67]:
exporter = CSVLineupExporter(optimizer.optimize(n=20, max_exposure=.25))
exporter.export('submissions/Week-{}/fanduel-lineups-qbte.csv'.format(current_week))